# Data Processing 
In this project data from the Brussels Mobility Bike Counts API will be processed using PySpark. The devices are sensors located in Brussels to count the number of bikes passed by and measure their speed every 15 minutes. The data that will be used is historical data from 2018-12-06 to 2023-
03-31. This can be done by reading csv from the URL provided.

In [1]:
# importing libraries
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F
from pyspark.sql.functions import col

from pyspark.sql.types import StructField, StringType, StructType, IntegerType, FloatType
import pandas as pd
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import requests

##### Downloading the data
- First, we gather the device names by querying the API, requesting the devices with `?request=devices` for all the required dates.
- Then, we only select the device names and remove duplicates by building a set.

In [2]:
devices_full_url = 'https://data.mobility.brussels/bike/api/counts/?request=devices&startDate=20181206&endDate=20230331&outputFormat=csv'
devices_full_df = pd.read_csv(devices_full_url)

device_names = set(devices_full_df['Device name'])


It is seen that there are 18 sensors to be analyzed in the project. To download the data, we build a list from the set and iterate over the list. The aim is to store all the data in a single `data.csv` file. So we first open the file, and then we iterate over all the devices. For each, we make a request through the API and write the result in the big file.

In [3]:
# Download data from the 18 sensors and store it in a single file `data.csv`
!mkdir SENSORS_DATA
with open(f'SENSORS_DATA/data.csv', 'w', newline='') as f :
  writer = csv.writer(f)
  for device in list(device_names) :

      # Formulate the URL
      url = 'https://data.mobility.brussels/bike/api/counts/?request=history&featureID='+device+'&startDate=20181206&endDate=20230331&outputFormat=csv'
      # Get the results from the request
      response = requests.get(url)
      # Format the answer
      lines = response.text.splitlines()
      header = lines[0] + ",Sensor"
      
      # The header has been intentionnally removed.
      # The structure of the files is `Date,Time gap,Count,Average speed,Sensor`
      for line in lines[1:] :
          line += f",{device}"
          writer.writerow(line.split(','))

# Filling missing data
It is seen that some sensors have several missing time stamps, so to complete the process they need to be added. Each day has 96 time gaps. A function to flatten the time gap by computing the number of days between 2018-12-06 and 2023-03-31 and setting time gap of 0 to the starting date then adding 96 to each day. By setting the time gap as index then reindexing the data we will have all the missing data and they can filled with count 0 and speed -1.

In [4]:
# FLATTEN TIME GAP
# A pair 'day, time gap' is mapped to an absolute timegap, with timestamp 0 being '2018-12-06, 0'. 
# Each day is 96 time gaps.
# So we first compute the number of days between '2018-12-06' and the current day
# And we multiply this by 96 to have the number of elapsed time gaps
# And we add the time gap of the day, which is the other element in the pair
gaps_per_day = 96
initialDayString = '2018-12-06'
initialDay = datetime.strptime(initialDayString, '%Y-%m-%d')

# Requires tgap to be integer, and 'day' to be a string 
def flatten_timestamp(day, tgap) :
    day_delta = (datetime.strptime(day,'%Y-%m-%d') -initialDay).days
    return tgap + day_delta*gaps_per_day

# Works also in a single line
# flatten_timestamp = lambda tgap : int(tgap[1]) + (datetime.strptime(str(tgap[0]),'%Y-%m-%d') -initialDay).days*gaps_per_day



In [5]:
# Min and max timestamp
max_timestamp = flatten_timestamp('2023-03-31', 96)
min_timestamp = 0
timestamp_range = range(min_timestamp, max_timestamp)

In [6]:
# reading the csv and setting the columns
df = pd.read_csv("SENSORS_DATA/data.csv", header=None)
df.columns = ["day", "tgap","Count", "Speed", "DeviceName"]
deviceNames = df["DeviceName"].drop_duplicates().values

df.head()

,day,tgap,Count,Speed,DeviceName
0,2018-12-06,51,3,21,CJM90
1,2018-12-06,52,23,19,CJM90
2,2018-12-06,53,23,19,CJM90
3,2018-12-06,54,12,17,CJM90
4,2018-12-06,55,13,21,CJM90


In [7]:
!mkdir CLEAN_DATA
i=0
for device in deviceNames :
  i += 1
  print(f"Processing {i}")
  
  df_device = df[df["DeviceName"] == device]
  print(f"Dataset for sensor {device} loaded")

  # Extracting all the time gaps from the dataframe in a single columnar dataframe
  gaps_df = df_device.apply(lambda x : flatten_timestamp(x[0], x[1]), axis=1)

  # Setting this column as the index of the dataframe
  df_device.index = gaps_df

  # Removing initial 'day, time gap' columns
  df_device = df_device[["Count","Speed","DeviceName"]]

  # Time to fill missing lines with Pandas's 'reindex' method
  # 'reindex' method can only fill with one value. Choosing 'np.NAN' so we
  # can later fill np.NAN with our values
  df_device = df_device.reindex(list(timestamp_range),fill_value=np.NAN)
  
  # Filling NANs accordingly
# 0 count 
# -1 speed
  df_device = df_device.fillna(value={
      "Count":0,
      "Speed":-1,
      "DeviceName":device})
# saving data files to CLEAN_DATA folder
  df_device.to_csv(f"CLEAN_DATA/{device}_v2.csv", header=False)

Processing 1
Dataset for sensor CJM90 loaded
Processing 2
Dataset for sensor CLW239 loaded
Processing 3
Dataset for sensor CEK18 loaded
Processing 4
Dataset for sensor CB1142 loaded
Processing 5
Dataset for sensor CEE016 loaded
Processing 6
Dataset for sensor CAT17 loaded
Processing 7
Dataset for sensor CB1699 loaded
Processing 8
Dataset for sensor CEK31 loaded
Processing 9
Dataset for sensor CEV011 loaded
Processing 10
Dataset for sensor CJE181 loaded
Processing 11
Dataset for sensor CB2105 loaded
Processing 12
Dataset for sensor CB1599 loaded
Processing 13
Dataset for sensor CVT387 loaded
Processing 14
Dataset for sensor CB1101 loaded
Processing 15
Dataset for sensor CEK049 loaded
Processing 16
Dataset for sensor CB02411 loaded
Processing 17
Dataset for sensor CB1143 loaded
Processing 18
Dataset for sensor COM205 loaded


# Starting a Spark session

In [2]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.executor.memory", "8G") \
    .appName("Batchprocessing") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
# Building the dataset of all sensor measures as an RDD
device = sc.textFile("CLEAN_DATA/*")
# Splitting by comma and casting to specific types for each column
device = device.map(lambda line : [int(line.split(',')[0]),float(line.split(',')[1]),float(line.split(',')[2]),line.split(',')[3]])

The data in the files is **structured**, so we want to specify a schema. The task is about computing analytics, which requires aggregation operations, so we chose a **Spark DataFrame** as data structure (more suitable for aggregated queries than RDDs).

In [4]:
# Defining the schema
schema = StructType([ \
    StructField("tgap",IntegerType(),True), \
    StructField("Count",FloatType(),True), \
    StructField("Speed",FloatType(),True), \
    StructField("DeviceName", StringType(), True), \
  ])
df = device.toDF(schema=schema)

In [5]:
# (optional) Verification of number of sensors considered

# measures = 151392 # 151392 measures per device file.
# # num = df.count()/measures
# num2 = df.select("DeviceName").distinct().count()
# print(num2)

18


To compute analytics, we rely on the `pyspark.sql.Window` object.

We also use `pyspark.sql.functions`, for instance to compute the sum of a column (`F.sum`), ...

For each row, a window will be defined with the `rangeBetween` function, whose first argument is the lower bound of the window and the second argument is the upper bound.
- Lower bound will be up to the first row of the dataframe, meaning the window takes "everything that is before"
- Upper bound will be zero, so that no other rows after the timegap is considered

The window also partitions by `DeviceName` for obvious reasons, because the analytics are to be computer for each device separately.

In [6]:
# Defining the window
deviceWindow = Window.orderBy('tgap').partitionBy("DeviceName")\
                    .rangeBetween(Window.unboundedPreceding, 0)

### Computing the moving average
The moving average is defined as $$\bar{c_i}(t) = 1/t \sum_{n = 0}^t c_i(n)$$

We compute it by first computing for each row the cumulative count by using the `F.sum` function over the window defined above.

Then, we divide the cumulative sum by the time gap `t`

In [7]:
# Computing the average
## First cumulative sum
df = df.withColumn('cum_sum', F.sum('Count').over(deviceWindow))
## Then divide by the time gap, and drop the cumulative sum column
df_analytics = df.withColumn('avg', df["cum_sum"]/(df["tgap"])).drop('cum_sum')

### Computing the standard deviation
The definition of the standard deviation is

$$ \sigma_i(t) = \sqrt{\sum_{n=0}^t [c_i(n) - \bar{c_i}(t)]^2}$$

A first approach would be to compute for each row the deviation by subtracting the count by the average, as
`df["Count] - df["avg"]`, but this would be $c_i(n)-\bar{c_i}(n)$, not $c_i(n)-\bar{c_i}(t)$.

To ensure we compute the right quantity, we used a trick to keep a track of the current row.

In [8]:
# Define the window of current row
current_row_window = Window.partitionBy("DeviceName").orderBy('tgap').rowsBetween(0, 0)
# Extract the average of the first row of the window, which is the current row
avg_current_row = F.first(F.col("avg")).over(current_row_window)

In [9]:
df_analytics = df_analytics.withColumn("stddev",
    F.sqrt(
        F.sum(
            F.pow(df_analytics["Count"]-avg_current_row, 2)     # Compute (c_i(n)-\bar c_i(t))^2
        ).over(window=deviceWindow)                             # Summing for all n
)                                                               # And taking the square root
)                           

In [21]:
# df_analytics.orderBy(F.desc("avg")).filter(df_analytics["tgap"] < 128999).show()

### Compute the possible pairs (sensor1, sensor2)
Using a simple join on the time gaps, and only keeping the pairs that have different sensor name by selecting `s1_Name < s2_Name` to ensure there are no duplicate pair `sensor1, sensor2 <-> sensor2, sensor1`

In [11]:
pairs_df = df_analytics.alias("sensor1").join(df_analytics.alias("sensor2"),\
                                   col("sensor1.tgap") == col("sensor2.tgap"))\
                                   .select(col("sensor1.tgap"),\
                                           col("sensor1.DeviceName").alias("s1_Name"),\
                                           col("sensor1.Count").alias("s1_Count"),\
                                           col("sensor1.avg").alias("s1_avg"),\
                                           col("sensor1.stddev").alias("s1_std"),\
                                           col("sensor2.DeviceName").alias("s2_Name"),\
                                           col("sensor2.Count").alias("s2_Count"),\
                                           col("sensor2.avg").alias("s2_avg"),\
                                           col("sensor2.stddev").alias("s2_std")\
                                   )\
                                   .where(col("s1_Name") < col("s2_Name"))\
                                   .orderBy("tgap")

### Computing the covariance
For each row (each pair, each time gap), we now compute the covariance, with a similar technique as for the standard deviation.

The covariance is defined as 

$$\sigma_{i,j}(t) = \sum_{n=0}^t (c_i(n)-\bar{c_i}(t)) \cdot (c_j(n)-\bar{c_j}(t))$$

For each row we will compute $(c_i(n)-\bar{c_i}(t)) \cdot (c_j(n)-\bar{c_j}(t))$, and sum over a window as usual. To compute this quantity and avoid computing $(c_i(n)-\bar{c_i}(\underline{n})) \cdot (c_j(n)-\bar{c_j}(\underline{n}))$, we will use the same trick as for computing the standard deviation.

In [12]:
# Defines a window limited to the current row
current_row_window = Window.partitionBy("s1_Name", "s2_Name").orderBy('tgap').rowsBetween(0, 0)

# Calculates the constant value of s1_avg and s2_avg for the current row
s1_avg_current_row = F.first(F.col("s1_avg")).over(current_row_window)
s2_avg_current_row = F.first(F.col("s2_avg")).over(current_row_window)

pairWindow = Window.partitionBy("s1_Name", "s2_Name").orderBy('tgap').rangeBetween(Window.unboundedPreceding, 0)

In [13]:
pairs_covariance = pairs_df.withColumn("covariance",
                                       F.sum(
                                        (F.col("s1_Count") - s1_avg_current_row) * (F.col("s2_Count") - s2_avg_current_row)
                                       ).over(pairWindow))

### Computing the Pearson correlation coefficient
The pearson correlation coefficient between two sensors $i$ and $j$ is defined for each time gap as 

$$
r_{ij}(t) = \frac{\sigma_{i,j}(t)}{\sigma_i(t)\cdot \sigma_j(t)}\; ,
$$
where the $\sigma_i$ are the moving standard deviation for each sensors as computed above, and the covariance $\sigma_{i,j}$ is computed as above as well.

In [14]:
pairs_correl = pairs_covariance.withColumn("pearsonCoeff",
                                           col("covariance")/(col("s1_std")*col("s2_std"))
                                           )\
                                         .orderBy(F.desc("pearsonCoeff"))

In [15]:
pairs_correl=pairs_correl.drop("s1_Count","s1_avg","s1_std","s2_Count","s2_avg","s2_std","covariance")

In [16]:
pairs_correl.show()

+----+-------+-------+------------------+
|tgap|s1_Name|s2_Name|      pearsonCoeff|
+----+-------+-------+------------------+
|  51| CEK049|  CJM90|               1.0|
|  51|CB02411| CB2105|               1.0|
|  51| CB2105| COM205|               1.0|
|  51| CB2105| CEK049|               1.0|
|  51|CB02411| CEK049|               1.0|
|  51|  CJM90| COM205|               1.0|
|  51| CB2105|  CJM90|               1.0|
|  51|CB02411|  CJM90|               1.0|
|  51|CB02411| COM205|               1.0|
|  51| CEK049| COM205|               1.0|
| 440| CB1142| CB1143|               1.0|
|  52|CB02411| CEK049| 0.998231052109537|
|  52|CB02411| COM205|0.9964446914271927|
|  52| CB2105| COM205|0.9920137015954205|
|  53| CEK049|  CJM90|0.9902007318886551|
|  52| CEK049| COM205|0.9896730715427468|
|  54|CB02411| CEK049|0.9866565316667033|
|  54|CB02411| COM205|0.9861355712508804|
|  53|CB02411| CEK049|0.9847967175254008|
|  53|CB02411| COM205|0.9838657312018031|
+----+-------+-------+------------

### Compute the top correlated pairs
- Partition by tgap and order by Pearson correlation coefficient
- For each tgap, associate the rank to each row
- Only keep rows with rank $\leq 5$

In [17]:
# Define the window partitioned by time gap and ordered by correlation coefficient in descending order
window = Window.partitionBy("tgap").orderBy(col("pearsonCoeff").desc())

# Add a new column with the rank of each row within the window
ranked_df = pairs_correl.withColumn("rank", F.rank().over(window))

# Filter for the top-5 pairs of sensors for each time gap
top_5_pairs = ranked_df.filter(col("rank") <= 5).orderBy(F.desc("tgap"), F.desc("pearsonCoeff")) \
    .select("tgap", "s1_Name", "s2_Name", "pearsonCoeff")


top_5_pairs.show()

+------+-------+-------+------------------+
|  tgap|s1_Name|s2_Name|      pearsonCoeff|
+------+-------+-------+------------------+
|151391| CB2105|  CJM90|0.8352454561723874|
|151391| CB2105| CEK049|0.8172815203063362|
|151391| CEK049|  CJM90|0.8133373740042474|
|151391| CB1599| CEK049|0.8088956581175814|
|151391|  CAT17| CVT387|0.7985076772625357|
|151390| CB2105|  CJM90|0.8352459639225993|
|151390| CB2105| CEK049|0.8172827939849917|
|151390| CEK049|  CJM90|0.8133371912720383|
|151390| CB1599| CEK049|0.8088952868085363|
|151390|  CAT17| CVT387|0.7985073661930835|
|151389| CB2105|  CJM90|0.8352457135708057|
|151389| CB2105| CEK049|0.8172825203389336|
|151389| CEK049|  CJM90|0.8133368503200527|
|151389| CB1599| CEK049|0.8088949827441813|
|151389|  CAT17| CVT387|0.7985072370714433|
|151388| CB2105|  CJM90| 0.835245472079438|
|151388| CB2105| CEK049|0.8172822510309377|
|151388| CEK049|  CJM90|0.8133365203973153|
|151388| CB1599| CEK049|0.8088944294256768|
|151388|  CAT17| CVT387|0.798507